### Deliverable 1: Preprocessing the Data for a Neural Network

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

#  Import and read the charity_data.csv.
import pandas as pd 
df = pd.read_csv("listing_clean_with_Postal_codes_errors_removed.csv")
df.head()

,Unnamed: 0,Unnamed: 0.1,Address,Price,Beds,Baths,City,Area,Street,Description,Postal_Code
0,0,0,2323 Confederation Pkwy 1102,430000,2,2,Mississauga,Cooksville,Confederation Pkwy,Beautiful Bright And Spacious Condo Unit 2 Bed...,L5B 1N1
1,1,1,816 Lansdowne Ave 411,598000,2,1,Toronto,Dovercourt-Wallace Emerson-Junction,Lansdowne Ave,Large Renovated One Bedroom+Den 646 Sf! Great ...,M6H 3Z6
2,2,6,109 Lake St,750000,4,2,St. Catharines,NaN,NaN,Investors!! Maintenance-Free Solid 4 Plex-Full...,L2R 5Y5
3,3,7,321 Queenston Rd,1899000,4,4,Niagara-on-the-Lake,NaN,NaN,"Welcome To 321 Queenston Rd, Overlooking Viney...",L8K 3P3
4,4,8,650 Upper Sherman Ave,899000,4,2,Hamilton,Burkholme,Upper Sherman Ave,Absolutely Fantastic Opportunity !!! Detach Ho...,L8V 3M7


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#  YOUR CODE GOES HERE

df = df[['Price','Beds','Baths','City']]

In [3]:
# Determine the number of unique values in each column.
#  YOUR CODE GOES HERE

df.nunique()

Price    1109
Beds       14
Baths      12
City      321
dtype: int64

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
#  YOUR CODE GOES HERE


application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
application_type_counts

In [ ]:
# Visualize the value counts of APPLICATION_TYPE
#  YOUR CODE GOES HERE

application_type_counts.plot.density()

In [ ]:
# Determine which values to replace if counts are less than ...?
#  YOUR CODE GOES HERE

replace_application = list(application_type_counts[application_type_counts < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

In [ ]:
# Look at CLASSIFICATION value counts for binning
#  YOUR CODE GOES HERE


classification_value_counts = application_df['CLASSIFICATION'].value_counts()
classification_value_counts

In [ ]:
# Visualize the value counts of CLASSIFICATION
#  YOUR CODE GOES HERE

classification_value_counts.plot.density()

In [ ]:
# Determine which values to replace if counts are less than ..?
#  YOUR CODE GOES HERE

replace_class = list(classification_value_counts[classification_value_counts < 800].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

In [ ]:
# Generate our categorical variable lists
#  YOUR CODE GOES HERE

application_cat= application_df.dtypes[application_df.dtypes =="object"].index.tolist()
application_cat

In [4]:
X = df[['Beds','Baths','City']].copy()
X['City'] = LabelEncoder().fit_transform(X['City'])
X = X.dropna()
X.head()

,Beds,Baths,City
0,2,2,182
1,2,1,287
2,4,2,264
3,4,4,195
4,4,2,120


In [ ]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
#  YOUR CODE GOES HERE

encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))


# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

In [ ]:
# Merge one-hot encoded features and drop the originals
#  YOUR CODE GOES HERE

application_df = application_df.merge(encode_df, left_index=True,right_index=True)
application_df = application_df.drop(columns = application_cat)
application_df.head()

In [5]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE

y = df['Price'].values
X = X.values


# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [6]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Deliverable 2: Compile, Train and Evaluate the Model

In [7]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 80
hidden_nodes_layer3 = 80

nn = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
#  YOUR CODE GOES HERE

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
#  YOUR CODE GOES HERE

nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                320       
                                                                 
 dense_1 (Dense)             (None, 80)                6480      
                                                                 
 dense_2 (Dense)             (None, 80)                6480      
                                                                 
 dense_3 (Dense)             (None, 1)                 81        
                                                                 
Total params: 13,361
Trainable params: 13,361
Non-trainable params: 0
_________________________________________________________________


In [8]:
# Compile the model
#  YOUR CODE GOES HERE

import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics= ["accuracy"])


In [9]:
# Train the model
#  YOUR CODE GOES HERE

cp_callback= ModelCheckpoint(filepath=checkpoint_path,save_weights_only=True, save_freq=5)
fit_model= nn.fit(X_train_scaled, y_train, epochs=500, verbose=1, callbacks=[cp_callback])

Epoch 1/500
77/77 [==============================] - 1s 8ms/step - loss: -36878388.0000 - accuracy: 0.0000e+00
Epoch 2/500
77/77 [==============================] - 0s 4ms/step - loss: -1453866496.0000 - accuracy: 0.0000e+00
Epoch 3/500
77/77 [==============================] - 0s 4ms/step - loss: -12710231040.0000 - accuracy: 0.0000e+00
Epoch 4/500
77/77 [==============================] - 0s 3ms/step - loss: -55344324608.0000 - accuracy: 0.0000e+00
Epoch 5/500
77/77 [==============================] - 0s 5ms/step - loss: -163292495872.0000 - accuracy: 0.0000e+00
Epoch 6/500
77/77 [==============================] - 0s 5ms/step - loss: -388085612544.0000 - accuracy: 0.0000e+00
Epoch 7/500
77/77 [==============================] - 0s 5ms/step - loss: -789527592960.0000 - accuracy: 0.0000e+00
Epoch 8/500
77/77 [==============================] - 0s 4ms/step - loss: -1432768217088.0000 - accuracy: 0.0000e+00
Epoch 9/500
77/77 [==============================] - 0s 4ms/step - loss: -2376279785472

77/77 [==============================] - 1s 8ms/step - loss: -30162477283016704.0000 - accuracy: 0.0000e+00
Epoch 133/500
77/77 [==============================] - 1s 7ms/step - loss: -30892928813498368.0000 - accuracy: 0.0000e+00A: 0s - loss: -30890459207303168.0000 - accuracy: 0.0000e
Epoch 134/500
77/77 [==============================] - 0s 6ms/step - loss: -31629315988783104.0000 - accuracy: 0.0000e+00
Epoch 135/500
77/77 [==============================] - 1s 7ms/step - loss: -32383026914656256.0000 - accuracy: 0.0000e+00
Epoch 136/500
77/77 [==============================] - 0s 6ms/step - loss: -33150080156434432.0000 - accuracy: 0.0000e+00
Epoch 137/500
77/77 [==============================] - 0s 5ms/step - loss: -33930426321993728.0000 - accuracy: 0.0000e+00
Epoch 138/500
77/77 [==============================] - 0s 5ms/step - loss: -34715642980466688.0000 - accuracy: 0.0000e+00
Epoch 139/500
77/77 [==============================] - 1s 7ms/step - loss: -35514725940854784.0000 - ac

77/77 [==============================] - 0s 6ms/step - loss: -277637474971287552.0000 - accuracy: 0.0000e+00
Epoch 263/500
77/77 [==============================] - 0s 5ms/step - loss: -281153369559531520.0000 - accuracy: 0.0000e+00
Epoch 264/500
77/77 [==============================] - 0s 5ms/step - loss: -284719034228801536.0000 - accuracy: 0.0000e+00
Epoch 265/500
77/77 [==============================] - 1s 9ms/step - loss: -288310777939492864.0000 - accuracy: 0.0000e+00
Epoch 266/500
77/77 [==============================] - 0s 6ms/step - loss: -291942842803159040.0000 - accuracy: 0.0000e+00
Epoch 267/500
77/77 [==============================] - 0s 6ms/step - loss: -295587414611591168.0000 - accuracy: 0.0000e+00
Epoch 268/500
77/77 [==============================] - 0s 6ms/step - loss: -299286309166383104.0000 - accuracy: 0.0000e+00
Epoch 269/500
77/77 [==============================] - 1s 7ms/step - loss: -302997298349080576.0000 - accuracy: 0.0000e+00
Epoch 270/500
77/77 [=========

77/77 [==============================] - 0s 6ms/step - loss: -1088831796457504768.0000 - accuracy: 0.0000e+00
Epoch 394/500
77/77 [==============================] - 0s 5ms/step - loss: -1098418988095897600.0000 - accuracy: 0.0000e+00
Epoch 395/500
77/77 [==============================] - 1s 7ms/step - loss: -1108045349836029952.0000 - accuracy: 0.0000e+00
Epoch 396/500
77/77 [==============================] - 0s 5ms/step - loss: -1117731841118306304.0000 - accuracy: 0.0000e+00
Epoch 397/500
77/77 [==============================] - 0s 4ms/step - loss: -1127480042490691584.0000 - accuracy: 0.0000e+00
Epoch 398/500
77/77 [==============================] - 0s 4ms/step - loss: -1137197457537499136.0000 - accuracy: 0.0000e+00
Epoch 399/500
77/77 [==============================] - 0s 6ms/step - loss: -1147003383270342656.0000 - accuracy: 0.0000e+00
Epoch 400/500
77/77 [==============================] - 0s 5ms/step - loss: -1157065563931934720.0000 - accuracy: 0.0000e+00
Epoch 401/500
77/77 [=

In [10]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

26/26 - 0s - loss: -2.1196e+18 - accuracy: 0.0000e+00 - 280ms/epoch - 11ms/step
Loss: -2.1196038814102979e+18, Accuracy: 0.0
